In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/kaggle/input/fake-news/train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


***WE Will try only with title and in second we will try both***

In [5]:
df = df.dropna()

In [6]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
X = df.drop('label', axis = 1)

In [8]:
y = df['label']

In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
from tensorflow.keras.layers import Embedding

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [13]:
from tensorflow.keras.preprocessing.text import one_hot

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [15]:
voc_size = 5000

**ONE HOT**

In [16]:
message = X.copy()

In [17]:
message.reset_index(inplace = True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [22]:
corpus = []
for i in range(0, len(message)):
   
    review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    

In [23]:
len(corpus)

18285

In [24]:
message['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [25]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

**One Hot**

In [26]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]

In [27]:
onehot_repr[0]

[4652, 676, 2990, 4162, 2766, 2182, 2485, 3426, 4928, 2303]

In [28]:
sent_length = 20
embeded_docs = pad_sequences(onehot_repr, padding = "pre", maxlen = sent_length)

In [29]:
embeded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4652,
        676, 2990, 4162, 2766, 2182, 2485, 3426, 4928, 2303], dtype=int32)

In [30]:
dims = 40
model = Sequential()

2022-03-02 22:36:51.731067: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [31]:
model.add(Embedding(voc_size, dims, input_length = sent_length))

In [32]:
model.add(LSTM(100))
model.add(Dense(1, activation = "sigmoid"))

In [33]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
X_final = np.array(embeded_docs)

In [35]:
y_final = np.array(y)

In [36]:
X_final.shape

(18285, 20)

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [39]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

2022-03-02 22:36:53.815513: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
192/192 [==============================] - 14s 57ms/step - loss: 0.3382 - accuracy: 0.8387 - val_loss: 0.1971 - val_accuracy: 0.9162
Epoch 2/10
192/192 [==============================] - 10s 53ms/step - loss: 0.1399 - accuracy: 0.9451 - val_loss: 0.1987 - val_accuracy: 0.9162
Epoch 3/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0973 - accuracy: 0.9640 - val_loss: 0.2338 - val_accuracy: 0.9165
Epoch 4/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0712 - accuracy: 0.9749 - val_loss: 0.2803 - val_accuracy: 0.9148
Epoch 5/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0503 - accuracy: 0.9827 - val_loss: 0.2931 - val_accuracy: 0.9080
Epoch 6/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0353 - accuracy: 0.9890 - val_loss: 0.3887 - val_accuracy: 0.9100
Epoch 7/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0214 - accuracy: 0.9934 - val_loss: 0.4186 - val_accuracy: 0.9

In [40]:
y_pred = model.predict(X_test)

In [41]:
from sklearn.metrics import accuracy_score